In [ ]:
import XLSX
xf = XLSX.readxlsx("datasett.xlsx")

In [ ]:
using DataFrames, XLSX
df1 = DataFrame(XLSX.readtable("datasett.xlsx", "t"))
df2 = DataFrame(XLSX.readtable("datasett.xlsx", "k"))
df3 = DataFrame(XLSX.readtable("datasett.xlsx", "i"))
df4 = DataFrame(XLSX.readtable("datasett.xlsx", "n"))
df5 = DataFrame(XLSX.readtable("datasett.xlsx", "o"))

In [ ]:
using JuMP, GLPK ,CPLEX
m = Model(CPLEX.Optimizer)

In [ ]:
t = select(df1, :t) # time index
T = nrow(df1)
P = select(df1, :P)
S = select(df1, :S)

In [ ]:
P[:,1] = convert(Vector{Float64}, df1[1:49,:P]) # time t의 구매가격 
S[:,1] = convert(Vector{Float64}, df1[1:49,:S]) # time t의 판매가격

In [ ]:
n = select(df4, :n) # charger index
N = nrow(df4)
α = select(df4, :α) # charging power
β = select(df4, :β) # discharging power
ρ_ch = select(df4, :ρ_ch) # charger n의 충전 효율
ρ_dis = select(df4, :ρ_dis) # charger n의 방전 효율

In [ ]:
α[:,1] = convert(Vector{Float64}, df4[1:8,:α]) # charger n의 충전power
β[:,1] = convert(Vector{Float64}, df4[1:8,:β]) # charger n의 방전power
ρ_ch[:,1] = convert(Vector{Float64}, df4[1:8,:ρ_ch]) # charger n의 충전 효율
ρ_dis[:,1] = convert(Vector{Float64}, df4[1:8,:ρ_dis]) # charger n의 방전 효율

In [ ]:
k = select(df2, :k) # bus index
K = nrow(df2)
C_bat = 88 # bus k의 배터리 capacity
E_0 = 0.5 # bus k의 초기 에너지 수준
E_min = 0.2 # bus k의 최소 에너지 수준
E_max = 0.8 # bus k의 최대 에너지 수준
E_end = 0.2 # bus k의 하루 운행 종료 후 최소 에너지 수준
U = 135 # transformer capa

In [ ]:
i = select(df3, :i) # trip index
I = nrow(df3)
T_start = select(df3, :T_start) # time t에 출발
T_end = select(df3, :T_end) # time t에 도착
γ = select(df3, :γ) # 시간당 trip i의 평균 에너지 소비 수준

In [ ]:
T_start[:,1] = convert(Vector{Int}, df3[1:27,:T_start]) # trip i의 시작
T_end[:,1] = convert(Vector{Int}, df3[1:27,:T_end]) # trip i의 종료
γ[:,1] = convert(Vector{Float64}, df3[1:27,:γ]) # trip i의 consumption

In [ ]:
@variable(m, b[1:K,1:I,1:T], Bin) # bus k가 trip i를 serving 하는지 여부 
@variable(m, x[1:K,1:N,1:T], Bin) # bus k가 charger n에서 time t에 charging 여부
@variable(m, y[1:K,1:N,1:T], Bin) # bus k가 charger n에서 time t에 discharging 여부
@variable(m, c[1:K,1:T], Bin) # bus k의 time t에 charging, discharging 여부

In [ ]:
@variable(m, e[1:K,1:T]>=0) # time t에서 bus k의 에너지 수준 
@variable(m, w_buy[1:T]>=0) # time t에 충전량
@variable(m, w_sell[1:T]>=0) # time t에 방전량

In [ ]:
@objective(m, Max, sum(S[t,1] * w_sell[t] for t in 1:T) - sum(P[t,1] * w_buy[t] for t in 1:T))

In [ ]:
@constraint(m, c1[k in 1:K, t in 1:T], sum(b[k ,i, t] for i in 1:I) + sum(x[k ,n, t] for n in 1:N) + sum(y[k ,n, t] for n in 1:N) <= 1)

In [ ]:
@constraint(m, c2[i in 1:I,k in 1:K, t in T_start[i,1]:T_end[i,1] - 1],b[k, i, t + 1] >= b[k, i ,t])

In [ ]:
@constraint(m, c3[i in 1:I, t in T_start[i,1]:T_end[i,1]],sum(b[k, i, t] for k in 1:K ) == 1)

In [ ]:
@constraint(m, c4[n in 1:N, t in 1:T], sum(x[k,n,t] for k in 1:K) + sum(y[k,n,t] for k in 1:K) <= 1)

In [ ]:
@constraint(m, c5[k in 1:K, t in 2:T], e[k,t] == e[k,t-1] + sum(ρ_ch[n,1]*α[n,1]*x[k,n,t] for n in 1:N)- sum(γ[i,1]*b[k,i,t] for i in 1:I) - sum(1/ρ_dis[n,1]*β[n,1]*y[k,n,t] for n in 1:N))

In [ ]:
@constraint(m, c6[t in 1:T], sum(sum(α[n,1] * x[k,n,t] for k in 1:K) for n in 1:N) == w_buy[t])

In [ ]:
@constraint(m, c7[t in 1:T], sum(sum(β[n,1] * x[k,n,t] for k in 1:K) for n in 1:N) == w_sell[t])

In [ ]:
@constraint(m, c8[k in 1:K], e[k,1] == E_0 * C_bat)

In [ ]:
@constraint(m, c9[k in 1:K, t in 1:T], C_bat * E_min <= e[k,t] <= C_bat * E_max)

In [ ]:
@constraint(m, c10[t in 1:T], sum(sum(α[n,1] * x[k,n,t] for k in 1:K) for n in 1:N) + sum(sum(β[n,1] * y[k,n,t] for k in 1:K) for n in 1:N) <= U)

In [51]:
JuMP.optimize!(m)

println("b[1:K,1:I,1:T] = ", JuMP.value.(b))
println("x[1:K,1:N,1:T] = ", JuMP.value.(x))
println("y[1:K,1:N,1:T] = ", JuMP.value.(y)) 
println("c[1:K,1:T] = ", JuMP.value.(c))
println("e[1:K,1:T] = ", JuMP.value.(e))
println("w_buy[1:T] = ", JuMP.value.(w_buy))
println("w_sell[1:T] = ", JuMP.value.(w_sell))

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.05 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   = 7023.72 sec.
  Wait time (average)   =    0.09 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.05 ticks)

Consider using CPLEX node files to reduce memory usage.
CPLEX Error  1001: Out of memory.
b[1:K,1:I,1:T] = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

x[1:K,1:N,1:T] = [1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0

 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 -0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 

y[1:K,1:N,1:T] = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 

0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0;;; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 -0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0;;; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0

e[1:K,1:T] = [44.0 18.11764705882353 36.817647058823525 43.107647058823524 34.02364705882353 24.939647058823528 43.63964705882353 34.555647058823524 25.471647058823525 44.171647058823524 62.87164705882353 36.989294117647056 55.68929411764705 46.60529411764705 37.52129411764705 28.437294117647053 47.13729411764705 38.053294117647056 56.75329411764706 63.04329411764706 53.95929411764706 44.87529411764706 35.791294117647055 54.49129411764706 28.608941176470587 47.30894117647058 66.00894117647059 40.126588235294115 58.82658823529411 32.94423529411764 51.644235294117635 25.761882352941164 44.46188235294116 35.37788235294116 26.293882352941157 44.99388235294116 63.69388235294116 54.609882352941156 28.727529411764685 47.42752941176468 66.12752941176468 57.04352941176468 47.95952941176468 66.65952941176468 40.77717647058821 59.477176470588205 50.3931764705882 69.0931764705882 69.0931764705882; 44.0 18.11764705882353 36.817647058823525 55.51764705882353 46.433647058823524 65.13364705882353 56.0